# Cleanup
This notebook is provided to help you clean up any resources you have created by running through the example. You should also go to the [CloudFormation console](https://console.aws.amazon.com/cloudformation/home) and delete the stack that you created.

In [9]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import json
import sys

# role = get_execution_role()
role = 'arn:aws:iam::499469212086:role/sagemaker_creditcard'

sm = boto3.Session().client(service_name='sagemaker')

### Delete feature groups

In [12]:
try:
    sm.delete_feature_group(FeatureGroupName='cc-agg-batch-fg') 
    print('deleted batch fg')
except:
    pass

try:
    sm.delete_feature_group(FeatureGroupName='cc-agg-fg') # use if needed to re-create
    print('deleted fg')
except:
    pass

In [13]:
sm.list_feature_groups()

{'FeatureGroupSummaries': [],
 'ResponseMetadata': {'RequestId': '2b98b606-5937-4507-a431-66c37379f8e1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2b98b606-5937-4507-a431-66c37379f8e1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '28',
   'date': 'Mon, 29 May 2023 00:17:54 GMT'},
  'RetryAttempts': 0}}

### Stop the KDA SQL App

In [14]:
import boto3
kda_client = boto3.client('kinesisanalytics')

try:
    kda_client.stop_application(ApplicationName='cc-agg-app')
except:
    pass

print('Stopped the KDA SQL app')

Stopped the KDA SQL app


### Delete the KDA SQL App

In [15]:
import time
try:
    ready = False
    while not ready:
        app_desc = kda_client.describe_application(ApplicationName='cc-agg-app')['ApplicationDetail']
        if app_desc['ApplicationStatus'] == 'READY':
            ready = True
        else:
            print('Waiting for KDA SQL app to be ready for deletion...')
            time.sleep(15)
    create_timestamp = app_desc['CreateTimestamp']
    response = kda_client.delete_application(ApplicationName='cc-agg-app',
                                  CreateTimestamp=create_timestamp)
    print('Deleted KDA SQL app')
except:
    print('FAILED to delete KDA sql app')

Waiting for KDA SQL app to be ready for deletion...
Deleted KDA SQL app


### Remove the trigger from Lambda

In [16]:
import boto3

%store -r 

lambda_client = boto3.client('lambda')
paginator = lambda_client.get_paginator('list_event_source_mappings')
mapping_iterator = paginator.paginate(FunctionName=lambda_to_model_arn)

for m in mapping_iterator:
    if len(m['EventSourceMappings']) > 0:
        uuid = m['EventSourceMappings'][0]['UUID']
        print(f'Deleting mapping: {uuid}...')
        lambda_client.delete_event_source_mapping(UUID=uuid)

Deleting mapping: a3c5f978-c25b-4fad-a85a-74e146019877...


### Delete the Kinesis data stream

In [17]:
kinesis_client = boto3.client('kinesis')
try:
    kinesis_client.delete_stream(StreamName='cc-stream')
except:
    pass
print('deleted Kinesis stream')

deleted Kinesis stream


### Delete the SageMaker endpoint

In [18]:
%store -r
try:
    sm.delete_endpoint(EndpointName=endpoint_name)
except:
    pass